# Artificial Neural Networks and Deep Learning

---

## Homework 2: Minimal Working Example

To make your first submission, follow these steps:
1. Create a folder named `[2024-2025] AN2DL/Homework 2` in your Google Drive.
2. Upload the `mars_for_students.npz` file to this folder.
3. Upload the Jupyter notebook `Homework 2 - Minimal Working Example.ipynb`.
4. Load and process the data.
5. Implement and train your model.
6. Submit the generated `.csv` file to Kaggle.


## 🌐 Connect Colab to Google Drive

In [7]:
from google.colab import drive

drive.mount("/gdrive")
%cd /gdrive/My Drive/[2024-2025] AN2DL/Homework 2

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/[2024-2025] AN2DL/Homework 2


## ⚙️ Import Libraries

In [8]:
import os
from datetime import datetime

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
from tensorflow.keras import models
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tfk.__version__}")
print(f"GPU devices: {len(tf.config.list_physical_devices('GPU'))}")

TensorFlow version: 2.17.1
Keras version: 3.5.0
GPU devices: 1


## ⏳ Load the Data

In [9]:
data = np.load("mars_for_students.npz")

training_set = data["training_set"]
X_train = training_set[:, 0]
y_train = training_set[:, 1]

X_test = data["test_set"]

print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f"Test X shape: {X_test.shape}")

# Separiamo il train set in training e validation set
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42  # 20% per il validation set
)

Training X shape: (2615, 64, 128)
Training y shape: (2615, 64, 128)
Test X shape: (10022, 64, 128)


## 🛠️ Train and Save the Model

In [10]:
# Add color channel and rescale pixels between 0 and 1
X_train = X_train[..., np.newaxis] / 255.0
###
X_val = X_val[..., np.newaxis] / 255.0
X_test = X_test[..., np.newaxis] / 255.0

input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

# Verifica della forma dei dati
print(f"Training X shape: {X_train.shape}")
print(f"Training y shape: {y_train.shape}")
print(f"Validation X shape: {X_val.shape}")
print(f"Validation y shape: {y_val.shape}")
print(f"Test X shape: {X_test.shape}")

print(f"Input shape: {input_shape}")
print(f"Number of classes: {num_classes}")

Training X shape: (2092, 64, 128, 1)
Training y shape: (2092, 64, 128)
Validation X shape: (523, 64, 128, 1)
Validation y shape: (523, 64, 128)
Test X shape: (10022, 64, 128, 1)
Input shape: (64, 128, 1)
Number of classes: 5


## Definisco modello

ho aggiunto dropout e provo a usare le LeakyReLU

In [11]:
def encoder_block(input_tensor, num_filters):
    # Primo layer convoluzionale
    x = tfkl.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.LeakyReLU(negative_slope=0.1)(x)
    x = tfkl.Dropout(0.3)(x)

    # Secondo layer convoluzionale
    x = tfkl.Conv2D(num_filters, (3, 3), padding='same')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.LeakyReLU(negative_slope=0.1)(x)
    x = tfkl.Dropout(0.3)(x)

    # MaxPooling per downsampling
    p = tfkl.MaxPooling2D((2, 2))(x)
    return x, p


In [12]:
def decoder_block(input_tensor, concat_tensor, num_filters):
    # Upsampling (upscaling) della feature map
    x = tfkl.Conv2DTranspose(num_filters, (3, 3), strides=(2, 2), padding='same')(input_tensor)
    # Concatenazione con l'output dell'encoder corrispondente
    x = tfkl.concatenate([x, concat_tensor])
    # Convoluzione dopo la concatenazione
    x = tfkl.Conv2D(num_filters, (3, 3), padding='same')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.LeakyReLU(negative_slope=0.1)(x)
    x = tfkl.Dropout(0.3)(x)

    x = tfkl.Conv2D(num_filters, (3, 3), padding='same')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.LeakyReLU(negative_slope=0.1)(x)
    x = tfkl.Dropout(0.3)(x)

    return x


In [15]:
def unet(input_shape=(64, 128, 1), num_classes=5):
    inputs = tfkl.Input(shape=input_shape)

    # Encoder (downsampling)
    e1, p1 = encoder_block(inputs, 64)
    e2, p2 = encoder_block(p1, 128)
    e3, p3 = encoder_block(p2, 256)
    e4, p4 = encoder_block(p3, 512)
    e5, p5 = encoder_block(p4, 1024)

    # Bottleneck (layer centrale)
    b1 = tfkl.Conv2D(1024, (3, 3), activation='relu', padding='same')(p5)
    b1 = tfkl.Conv2D(1024, (3, 3), activation='relu', padding='same')(b1)

    d1 = decoder_block(b1, e5, 1024)
    d2 = decoder_block(d1, e4, 512)
    d3 = decoder_block(d2, e3, 256)
    d4 = decoder_block(d3, e2, 128)
    d5 = decoder_block(d4, e1, 64)

    # Output layer with softmax activation for multi-class segmentation
    outputs = tfkl.Conv2D(num_classes, (1, 1), activation='softmax')(d5)

    # Crea il modello
    model = models.Model(inputs, outputs)

    return model

In [16]:
# Crea il modello U-Net
model = unet(input_shape=(64, 128, 1), num_classes=5)

# Define the MeanIoU ignoring the background class
mean_iou = tfk.metrics.MeanIoU(num_classes=num_classes, ignore_class=0, sparse_y_pred=False)

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=[mean_iou,'accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 64, 128, 1)     │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_12 (Conv2D)        │ (None, 64, 128, 64)    │            640 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_10    │ (None, 64, 128, 64)    │            256 │ conv2d_12[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_10            │ (None, 64, 128, 64)    │              0 │ batch_normalization_1… │
│ (LeakyReLU)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_10 (Dropout)      │ (None, 64, 128, 64)    │              0 │ leaky_re_lu_10[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_13 (Conv2D)        │ (None, 64, 128, 64)    │         36,928 │ dropout_10[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_11    │ (None, 64, 128, 64)    │            256 │ conv2d_13[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_11            │ (None, 64, 128, 64)    │              0 │ batch_normalization_1… │
│ (LeakyReLU)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_11 (Dropout)      │ (None, 64, 128, 64)    │              0 │ leaky_re_lu_11[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_5           │ (None, 32, 64, 64)     │              0 │ dropout_11[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_14 (Conv2D)        │ (None, 32, 64, 128)    │         73,856 │ max_pooling2d_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_12    │ (None, 32, 64, 128)    │            512 │ conv2d_14[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_12            │ (None, 32, 64, 128)    │              0 │ batch_normalization_1… │
│ (LeakyReLU)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_12 (Dropout)      │ (None, 32, 64, 128)    │              0 │ leaky_re_lu_12[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_15 (Conv2D)        │ (None, 32, 64, 128)    │        147,584 │ dropout_12[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1

 Total params: 91,172,421 (347.80 MB)

 Trainable params: 91,156,549 (347.73 MB)

 Non-trainable params: 15,872 (62.00 KB)

In [17]:
# Setup callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_mean_io_u',
    mode='max',
    patience=30,
    restore_best_weights=True
)

# Define Callbacks
callbacks = [ tf.keras.callbacks.EarlyStopping( monitor='val_mean_io_u', mode='max', patience=30, restore_best_weights=True),
              tf.keras.callbacks.ModelCheckpoint("Tom_best_model.keras", save_best_only=True, monitor="val_mean_io_u"),
              tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5, min_lr=1e-6)
            ]

In [18]:
model.fit(
    X_train, y_train,  # Dati di addestramento
    epochs=200,         # Numero massimo di epoche
    batch_size=32,     # Dimensione del batch
    validation_data=(X_val, y_val),  # Set di validazione
    callbacks=callbacks
)

Epoch 1/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 87s 625ms/step - accuracy: 0.3287 - loss: 1.5509 - mean_io_u: 0.1405 - val_accuracy: 0.2032 - val_loss: 6.3452 - val_mean_io_u: 0.0546 - learning_rate: 0.0010
Epoch 2/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - accuracy: 0.4076 - loss: 1.2897 - mean_io_u: 0.1838 - val_accuracy: 0.2030 - val_loss: 3.2729 - val_mean_io_u: 0.0684 - learning_rate: 0.0010
Epoch 3/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.4555 - loss: 1.2024 - mean_io_u: 0.2180 - val_accuracy: 0.2135 - val_loss: 1.9104 - val_mean_io_u: 0.0828 - learning_rate: 0.0010
Epoch 4/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - accuracy: 0.5005 - loss: 1.1418 - mean_io_u: 0.2363 - val_accuracy: 0.2177 - val_loss: 2.2182 - val_mean_io_u: 0.0662 - learning_rate: 0.0010
Epoch 5/200
66/66 ━━━━━━━━━━━━━━━━━━━━ 4s 56ms/step - accuracy: 0.5354 - loss: 1.0924 - mean_io_u: 0.2591 - val_accuracy: 0.3406 - val_loss: 2.6159 - val_mean_io_u: 0.1147 - learning_rate: 0.0010
Epoch 6/200
66/66 

In [19]:
timestep_str = datetime.now().strftime("%y%m%d_%H%M%S")
model_filename = f"model_{timestep_str}.keras"
model.save(model_filename)
del model

print(f"Model saved to {model_filename}")

Model saved to model_241202_163423.keras


## 📊 Prepare Your Submission

In our Kaggle competition, submissions are made as `csv` files. To create a proper `csv` file, you need to flatten your predictions and include an `id` column as the first column of your dataframe. To maintain consistency between your results and our solution, please avoid shuffling the test set. The code below demonstrates how to prepare the `csv` file from your model predictions.




In [20]:
# If model_filename is not defined, load the most recent model from Google Drive
if model_filename not in globals() or model_filename is None:
    files = [f for f in os.listdir('.') if os.path.isfile(f) and f.startswith('model_') and f.endswith('.keras')]
    files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    if files:
        model_filename = files[0]
    else:
        raise FileNotFoundError("No model files found in the current directory.")

In [21]:
model = tfk.models.load_model(model_filename)
print(f"Model loaded from {model_filename}")

Model loaded from model_241202_163423.keras


In [22]:
preds = model.predict(X_test)
preds = np.argmax(preds, axis=-1)
print(f"Predictions shape: {preds.shape}")

314/314 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step
Predictions shape: (10022, 64, 128)


In [23]:
def y_to_df(y) -> pd.DataFrame:
    """Converts segmentation predictions into a DataFrame format for Kaggle."""
    n_samples = len(y)
    y_flat = y.reshape(n_samples, -1)
    df = pd.DataFrame(y_flat)
    df["id"] = np.arange(n_samples)
    cols = ["id"] + [col for col in df.columns if col != "id"]
    return df[cols]

In [24]:
# Create and download the csv submission file
timestep_str = model_filename.replace("model_", "").replace(".keras", "")
submission_filename = f"submission_{timestep_str}.csv"
submission_df = y_to_df(preds)
submission_df.to_csv(submission_filename, index=False)

from google.colab import files
files.download(submission_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#  
<img src="https://airlab.deib.polimi.it/wp-content/uploads/2019/07/airlab-logo-new_cropped.png" width="350">

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/95/Instagram_logo_2022.svg/800px-Instagram_logo_2022.svg.png" width="15"> **Instagram:** https://www.instagram.com/airlab_polimi/

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/81/LinkedIn_icon.svg/2048px-LinkedIn_icon.svg.png" width="15"> **LinkedIn:** https://www.linkedin.com/company/airlab-polimi/
___
Credits: Alberto Archetti 📧 alberto.archetti@polito.it





```
   Copyright 2024 Alberto Archetti

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
```